In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

import pickle
import json
import sys
import numpy.random as random
sys.path.append("../../../")

import time 

from src.online_prediction import DataProcessor
from src.context_aware.models import createModel

In [4]:
configPath = "../../config/combined_flows_forward.json"
modelFolder = "../../../data/models/context_aware"
trafficDataFolder = "../../../data/processed/dpdr"
verbose = True

In [3]:
config = json.load(open(configPath))
name = config.get("NAME")
len_window = config.get("LEN_WINDOW")
dim_data =  len(config.get("CONTEXT_IDXS"))

In [6]:
csv_data = pd.read_csv(f"{trafficDataFolder}/{name}.csv")

with open(f"{modelFolder}/{name}_lenWindow_{len_window}_metaConfig.pkl", "rb") as f:
    metaConfig = pickle.load(f)

metaConfig.display()

MetaConfig:
  dim_data: 12
  window_length: 20
  history_length: 100
  smooth_fc: 1.5
  degree: 3
  Ts: 0.01
  min_vals: [-0.36894907  0.90292435 -0.38260963 -0.41950114  0.89724432 -0.50256594
 -0.44173692  0.89601395 -0.50351072 -0.37503843  0.99577361 -0.40302067]
  max_vals: [ 0.49172105  1.1641571  -0.03381354  0.52639037  1.1625885  -0.1044017
  0.52653422  1.13691496 -0.10175204  0.42422338  1.16192049 -0.034515  ]


In [14]:
dataProcessor = DataProcessor(metaConfig)
for i in range(metaConfig.window_length):
    data_point = csv_data.iloc[i, 2:].values
    dataProcessor.add_data_point(data_point)
    
context, last_trans_sources, context_no_smooth, (flags, timestamps_bin) = dataProcessor.get_window_features()
print(context.shape)
print(last_trans_sources.shape)
print(context_no_smooth.shape)



(20, 12)
(1, 12)
(20, 12)
